# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## 1. Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [2]:
df=pd.read_csv('users_behavior.csv')


In [3]:
df.head()

,Unnamed: 0,calls,minutes,messages,mb_used,is_ultra
0,0,40.0,311.90,83.0,19915.42,0
1,1,85.0,516.75,56.0,22696.96,0
2,2,77.0,467.66,86.0,21060.45,0
3,3,106.0,745.53,81.0,8437.39,1
4,4,66.0,418.74,1.0,14502.75,0


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Дата сет имеет 5 столбцов и 3214 строк.
Пропуской нет!
Столбцы:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Типы данных соответствуют.

## 2. Разбейте данные на выборки

In [17]:
# Разбиваю датасет на 3 выборки:обучающую, валидационную, тестовую
df_train, df_valid_test=train_test_split(df,train_size=0.6,test_size=0.4,random_state=1234)
df_valid,df_test=train_test_split(df_valid_test,test_size=0.5,random_state=1234)

In [18]:
len(df_train) # выборка для обучения

1928

In [19]:
len(df_valid) # выборка валидационная

643

In [20]:
len(df_test) # выборка тестовая

643

## 3. Исследуйте модели

На базе обучающей выборки формирую датасет признаков и serias целевого признака ['is_ultra']

In [32]:
features_train=df_train.drop(['is_ultra'],axis=1)
target_train=df_train['is_ultra']

In [33]:
df_features_train.head(5) # Признаки 

,calls,minutes,messages,mb_used
265,60.0,442.02,16.0,18306.45
1410,42.0,333.36,48.0,14566.34
728,43.0,298.42,12.0,18311.24
552,16.0,132.80,107.0,21893.35
3117,46.0,336.76,49.0,10710.72


In [34]:
df_target_train.head() # целевой признак

265     1
1410    0
728     0
552     1
3117    0
Name: is_ultra, dtype: int64

На базе валидационной выборки формирую датасет признаков и serias целевого признака ['is_ultra']

In [35]:
features_valid=df_valid.drop(['is_ultra'],axis=1) # Признаки 
target_valid=df_valid['is_ultra'] # целевой признак

На базе тестовой выборки формирую датасет признаков и serias целевого признака ['is_ultra']

In [36]:
features_test=df_valid.drop(['is_ultra'],axis=1) # Признаки 
target_test=df_valid['is_ultra'] # целевой признак

#### Модель "Дерево решений"

In [62]:
for depth in range(1,30,2):
    model_tree=DecisionTreeClassifier(random_state=1234, max_depth=depth) # присваиваем модели имя
    model_tree.fit(features_train,target_train) # обучаем
    prediction_tree=model_tree.predict(features_valid) # проверяем
    print('max_depth:',depth,end='')
    print(' accuracy:',accuracy_score(prediction_tree,target_valid))

max_depth: 1 accuracy: 0.776049766718507
max_depth: 3 accuracy: 0.8118195956454122
max_depth: 5 accuracy: 0.8087091757387247
max_depth: 7 accuracy: 0.8320373250388803
max_depth: 9 accuracy: 0.8180404354587869
max_depth: 11 accuracy: 0.8180404354587869
max_depth: 13 accuracy: 0.8040435458786936
max_depth: 15 accuracy: 0.7744945567651633
max_depth: 17 accuracy: 0.7480559875583204
max_depth: 19 accuracy: 0.744945567651633
max_depth: 21 accuracy: 0.7620528771384136
max_depth: 23 accuracy: 0.7387247278382582
max_depth: 25 accuracy: 0.7542768273716952
max_depth: 27 accuracy: 0.7542768273716952
max_depth: 29 accuracy: 0.7402799377916018


### Обучение прошло! При max_depth= 7 (лучшие значения гиперпараметров), качество модели "дерево предсказания"=0.83

#### Модель "Случайный лес"

In [88]:
for estim in range(1,50,2):    
    model_forest=RandomForestClassifier(max_depth=10,n_estimators=estim,random_state=1234,min_samples_leaf=5) # присваиваем модели имя
    model_forest.fit(features_train,target_train) # обучаем
    prediction_forest=model_forest.predict(features_valid) # проверяем
    print('n_estimators:',estim,end="")
    print(' accuracy:',accuracy_score(prediction_forest,target_valid))

n_estimators: 1 accuracy: 0.8055987558320373
n_estimators: 3 accuracy: 0.8149300155520995
n_estimators: 5 accuracy: 0.8320373250388803
n_estimators: 7 accuracy: 0.838258164852255
n_estimators: 9 accuracy: 0.8429237947122862
n_estimators: 11 accuracy: 0.8429237947122862
n_estimators: 13 accuracy: 0.8398133748055988
n_estimators: 15 accuracy: 0.8398133748055988
n_estimators: 17 accuracy: 0.838258164852255
n_estimators: 19 accuracy: 0.838258164852255
n_estimators: 21 accuracy: 0.8367029548989113
n_estimators: 23 accuracy: 0.8367029548989113
n_estimators: 25 accuracy: 0.8429237947122862
n_estimators: 27 accuracy: 0.838258164852255
n_estimators: 29 accuracy: 0.8351477449455676
n_estimators: 31 accuracy: 0.8351477449455676
n_estimators: 33 accuracy: 0.8351477449455676
n_estimators: 35 accuracy: 0.8351477449455676
n_estimators: 37 accuracy: 0.833592534992224
n_estimators: 39 accuracy: 0.833592534992224
n_estimators: 41 accuracy: 0.8351477449455676
n_estimators: 43 accuracy: 0.838258164852255


### Обучение прошло! При max_depth= 10, max_samples_leaf=5 и n_estimators= 9 (лучшее значение), качество модели "случайный лес"=0.84

#### Модель "Логистическая регрессия"

In [69]:
model_logistic_Reg=LogisticRegression() # присваиваем модели имя
model_logistic_Reg.fit(features_train,target_train) # обучаем
predict_LogisticReg=model_logistic_Reg.predict(features_valid) # проверяем
print('accuracy:',accuracy_score(predict_LogisticReg,target_valid))


accuracy: 0.7216174183514774


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### Обучение прошло! Модель не переобучивается. Качество модели "Логистическая регрессия"= 0.72. Самое низкое качество!

## Вывод:

Обучая три разный модели были полученны лучшие метрики качества:
    - Качество модели "Дерево предсказания" при гиперпараметрах max_depth= 7: 0.83
    - Качество модели "Случайный лес" при гиперпараметрах max_depth= 10, max_samples_leaf=5 и n_estimators= 9: 0.84
    - Качество модели "Логистическая регрессия": 0.72
    
Лучшей моделью получилась модель "Случайный лес"

## 4. Проверьте модель на тестовой выборке

#### Тестирование данных на модели "Дерево предсказаний"

In [93]:
model_tree=DecisionTreeClassifier(random_state=1234, max_depth=7)
model_tree.fit(features_train,target_train)
prediction_tree=model_tree.predict(features_test)
#print('max_depth:',depth,end='')
print(' accuracy:',accuracy_score(prediction_tree,target_test))

 accuracy: 0.8320373250388803


#### Тестирование данных на модели "Случайный лес"

In [97]:
model_forest=RandomForestClassifier(max_depth=10,n_estimators=9,random_state=1234,min_samples_leaf=5)
model_forest.fit(features_train,target_train)
prediction_forest=model_forest.predict(features_test)
#print('n_estimators:',estim,end="")
print(' accuracy:',accuracy_score(prediction_forest,target_test))

 accuracy: 0.8429237947122862


#### Тестирование данных на модели "Логистическая регрессия"

In [95]:
model_logistic_Reg=LogisticRegression()
model_logistic_Reg.fit(features_train,target_train)
predict_LogisticReg=model_logistic_Reg.predict(features_test)
print('accuracy:',accuracy_score(predict_LogisticReg,target_test))

accuracy: 0.7216174183514774


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


## Выводы:

Модели проверены на тестовой выборке. Метрики качества тестовой выборки=метрикам качества валидационной модели 